In [1]:
import json
import numpy as np
from scipy import sparse
import torch
import pickle
import heapq

In [2]:
question2idx = np.load('data/question2idx.npy',allow_pickle=True).item()
idx2question = np.load('data/idx2question.npy',allow_pickle=True).item()

In [3]:
model = torch.load(f='../alg/model-100/result.pt')
qq_table = sparse.load_npz('../alg/data/qq_table.npz').toarray()
qs_table = sparse.load_npz('../alg/data/qs_table.npz').toarray()
ss_table = sparse.load_npz('../alg/data/ss_table.npz').toarray()

In [4]:
num = 10 # 假设推荐五道
with open('test_history.json') as f:
    json_data = json.load(f)
history_answers = json_data['data']['exerciseRecordList'] # 获取历史答题记录

In [5]:
q_list = [question2idx[a['exerciseId']] for a in history_answers] # 获取答题记录的index
# 获取相关的习题
q_set_related = set()  # 所有相关问题的id集合
for q_id in q_list:
    q_set_related.update(np.where(qq_table[q_id] > 0)[0].tolist())
q_list_related = list(q_set_related)  # 所有相关问题的id数组
if num > len(q_list_related):  # 相关题目比需要推荐的少，允许[重复]选
    q_list_related *= (num / len(q_list_related) + 1)

In [8]:
len(set(q_list))

22

In [9]:
filtered_list = [item for item in q_list_related if item not in q_list]

In [10]:
len(filtered_list)

47

In [11]:
a_list = [1 if a['score'] > 0 else 0 for a in history_answers] # 获取答题记录的index

In [12]:
len(a_list)

100

In [38]:
max_length = 200
question_tensor_list = []
answer_tensor_list = []
mask_tensor_list = []
time_step = len(q_list)
for q_related in q_list_related:
    combined_q_list = q_list + [q_related] # 原始的答题记录加上最后一道用于预测的题目
    combined_a_list = a_list + [0]
    combined_m_list = [1 for i in range(len(combined_q_list))]
    if len(combined_q_list) < max_length:  # 如果序列长度小于最大长度
        combined_q_list += [0] * (max_length - len(combined_q_list))
        combined_a_list += [0] * (max_length - len(combined_a_list))
        combined_m_list += [0] * (max_length - len(combined_m_list))
    elif len(combined_q_list) > max_length:
        combined_q_list = combined_q_list[-max_length:]
        combined_a_list = combined_a_list[-max_length:]
        combined_m_list = combined_m_list[-max_length:]
    question_tensor_list.append(combined_q_list)
    answer_tensor_list.append(combined_a_list)
    mask_tensor_list.append(combined_m_list)
question_tensor = torch.tensor(question_tensor_list, dtype=torch.int64)
answer_tensor = torch.tensor(answer_tensor_list, dtype=torch.int64)
mask_tensor = torch.tensor(mask_tensor_list, dtype=torch.int64)

In [39]:
c_list = model(
        question=question_tensor,  
        response=answer_tensor,  
        mask=mask_tensor  
    ).squeeze(dim=0).tolist()

In [40]:
c_list[0]

[0.5,
 0.09329011291265488,
 0.99540114402771,
 0.9853528141975403,
 0.9999997615814209,
 0.9999986886978149,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.21285085380077362,
 2.468774198405299e-07,
 0.6484835147857666,
 0.9999624490737915,
 0.9999474287033081,
 0.974062979221344,
 0.0001970761950360611,
 0.0037434531841427088,
 0.7693697810173035,
 0.5354295372962952,
 0.9798581600189209,
 0.9999998807907104,
 1.0,
 1.0,
 0.9999998807907104,
 3.004030010855807e-11,
 9.931188742484665e-07,
 0.9999977350234985,
 1.0,
 0.9999998807907104,
 1.0,
 0.9999995231628418,
 0.9999996423721313,
 0.9999995231628418,
 0.9999995231628418,
 3.293131885584444e-05,
 1.7449110600864515e-07,
 0.02648080512881279,
 1.4119654224487022e-05,
 2.0622437659767456e-06,
 1.8434575110859441e-07,
 3.856914929656341e-07,
 2.840967283646023e-07,
 6.824170100117044e-07,
 6.777899557164346e-07,
 1.8115191835477162e-07,
 1.1440952363273027e-07,
 1.2217184064411413e-07,
 2.5056104391296685e-07,
 2.562811403095111e-07,
 2.69099

In [15]:
c_list[2][time_step]

0.9990842342376709

In [56]:
len(q_list_related)

68

In [67]:
len(q_list)

100

In [34]:
frequency = {}
for item in q_list:
    if frequency.get(item):
        frequency[item] = frequency[item]+1
    else:
        frequency[item] = 1
for item in filtered_list:
    frequency[item] = 0

In [76]:
len(set(q_list))

22

In [77]:
len(q_list_related)

68

In [78]:
len(filtered_list)

46

In [86]:
len(frequency)

68

In [35]:
predict = []
for i in range(len(q_list_related)):
    predict.append({
        'exerciseId': idx2question[q_list_related[i]],
        'predict': c_list[i][time_step-1],
        'isInHistory': 0 if q_list_related[i] in filtered_list else 1,
        'frequency': frequency[q_list_related[i]]
    })

In [64]:
len(predict)

68

In [103]:
len(q_list)

100

In [41]:
predict_list = [c_list[i][time_step] for i in range(len(q_list_related))]

In [19]:
len(predict_list)

69

In [42]:
recommend = heapq.nsmallest(num, zip(predict_list, q_list_related))
recommend

[(3.94132426348448e-12, 15),
 (4.937236036672843e-11, 136),
 (1.7563205290116457e-07, 127),
 (2.0057963467934314e-07, 1),
 (9.080370659830805e-07, 59),
 (9.591861953595071e-07, 171),
 (4.34222056355793e-06, 122),
 (1.0386169378762133e-05, 164),
 (1.3969324754725676e-05, 9),
 (5.104697265778668e-05, 144)]

In [56]:

c_list, q_list_recommend = [round(rec[0], 4) for rec in recommend], [idx2question[rec[1]] for rec in recommend]

In [57]:
q_list_recommend

[126, 1173, 1164, 112, 170, 1208, 1159, 1201, 120, 1181]

In [36]:
import pandas as pd
df = pd.DataFrame(predict, columns=['exerciseId', 'predict','isInHistory','frequency'])
df.to_excel('test_predict_with_answer_new1_timesep-1.xlsx')

In [49]:
s_set = set()  # 所有相关技能id集合
for q_id in q_list_recommend: # 这里为什么只更新推荐习题的相关知识点
    s_set.update(np.where(qs_table[q_id] > 0)[0].tolist())

In [53]:
qs_table.shape

(429, 145)

In [51]:
skill2idx = np.load('data/skill2idx.npy',allow_pickle=True).item()
idx2skill = np.load('data/skill2idx.npy',allow_pickle=True).item()

In [52]:
s_set


{5, 12, 13, 17, 18, 22, 29, 40, 46, 85, 95, 121}

In [ ]:
s_list = list(s_set)  # 相关技能的id数组
s_names = [s.name for s in Skill.query.filter(Skill.id.in_(s_list)).all()]  # 技能名称数组

In [ ]:

    # 计算推荐习题中技能的分布情况
   
    s_list = list(s_set)  # 相关技能的id数组
    ic(s_list)
    s_names = [s.name for s in Skill.query.filter(Skill.id.in_(s_list)).all()]  # 技能名称数组
    ic(s_names)
    s_q_num = [0 for _ in range(len(s_list))]  # 每个技能涉及的问题数量
    for q_id in q_list_recommend:
        s_list1 = np.where(qs_table[convert2Index(q_id)] > 0)[0].tolist()
        for s_id in s_list1:
            s_q_num[s_list.index(s_id)] += 1
    ic(s_q_num)
    s_values = [num / sum(s_q_num) for num in s_q_num]  # 除以和，使其元素之和为1
    return {
        'data': {
            'qList': q_list_recommend,
            'cList': c_list,
            'skillData': [{
                'value': value,
                'name': name
            } for name, value in zip(s_names, s_values)]
        }
    }


In [ ]:
num = int(request.args.get('num'))  # 推荐数量
    user_id = int(request.args.get('userId'))
    history_answers = Answer.query.filter_by(user_id=user_id).all()  # 该同学所有相关的问答
    q_list = [a.q_id for a in history_answers]  # 所有回答过的问题id
    ic(q_list)
    q_set_related = set()  # 所有相关问题的id集合
    for q_id in q_list:
        q_set_related.update(np.where(qq_table[q_id] > 0)[0].tolist())
    q_list_related = list(q_set_related)  # 所有相关问题的id数组
    if num > len(q_list_related):  # 相关题目比需要推荐的少，允许[重复]选
        q_list_related *= (num / len(q_list_related) + 1)


    ones = torch.ones(size=[1, len(q_list_related), ], dtype=torch.int, device=DEVICE)
    # TODO 2024/7/21 这里要改动，首先是模型加一个输出，其次是这里面的参数要改？
    c_list, knowledge_vectors = model(
        question=torch.unsqueeze(torch.tensor(q_list_related, device=DEVICE), dim=0),  # 问题id列表
        response=ones,  # 推荐时回答全设置为1
        mask=ones  # 题目都是有效的，maks也全为1
    ).squeeze(dim=0).tolist()
    # TODO 假设取前10个最小的 第一个参数为10
    recommend = heapq.nsmallest(10, zip(c_list, q_list))
    c_list, q_list_recommend = [round(rec[0], 4) for rec in recommend], [rec[1] for rec in recommend]
    # 计算推荐习题中技能的分布情况
    s_set = set()  # 所有相关技能id集合
    for q_id in q_list_recommend:
        s_set.update(np.where(qs_table[q_id] > 0)[0].tolist())
    s_list = list(s_set)  # 相关技能的id数组
    ic(s_list)
    s_names = [s.name for s in Skill.query.filter(Skill.id.in_(s_list)).all()]  # 技能名称数组
    ic(s_names)
    s_q_num = [0 for _ in range(len(s_list))]  # 每个技能涉及的问题数量
    for q_id in q_list_recommend:
        s_list1 = np.where(qs_table[q_id] > 0)[0].tolist()
        for s_id in s_list1:
            s_q_num[s_list.index(s_id)] += 1
    ic(s_q_num)
    s_values = [num / sum(s_q_num) for num in s_q_num]  # 除以和，使其元素之和为1

    # TODO 2024/7/21 用户协同过滤
    # 1. 获取当前用户的知识点掌握程度向量，或者从数据库里取
    current_user_vector = knowledge_vectors[user_id]

    # 2. 获取所有用户的知识点掌握程度向量 (这里需要从数据库或缓存中加载所有用户的knowledge_vectors;在gikt过程中把这个向量要存入数据库？)
    # all_knowledge_vectors = load_all_knowledge_vectors()  # 假设有这个函数
    all_knowledge_vectors = knowledge_vectors  # 从GIKT返回的 所有用户的 knowledge_vectors（GIKT返回的是所有的 还是一个batch里面的？）

    # 3. 获取当前用户的相似用户
    similar_users = get_similar_users(all_knowledge_vectors, current_user_vector, threshold=0.95) # 这个threshold可以根据情况调整

    # 4. 计算相似用户的平均知识点掌握程度
    avg_knowledge = average_knowledge(all_knowledge_vectors, similar_users)

    # 5. 根据 GIKT 模型得到的预备习题库进行过滤
    pre_recommend_list = q_list_recommend[:num]  # 假设模型返回了足够多的问题
    # emb_table_question.weight 模型内部的嵌入层权重，表示问题的嵌入向量
    question_similarities = torch.matmul(model.emb_table_question.weight[pre_recommend_list], avg_knowledge)
    filtered_recommended_questions = torch.topk(question_similarities, num).indices.tolist()

    return {
        'data': {
            # 'qList': q_list_recommend,
            # TODO 2024/07/21 修改 q_list_recommend ——> filtered_recommended_questions
            'qList': filtered_recommended_questions,
            'cList': c_list,
            'skillData': [{
                'value': value,
                'name': name
            } for name, value in zip(s_names, s_values)]
        }
    }

In [1]:
from ..test_kt_interface import fetch_exercise_records,stu_state_upload, cal_stu_knowledge_state_similarity

ImportError: attempted relative import with no known parent package